In [44]:
import time
from __future__ import division
from web3 import Web3, HTTPProvider
import json

import pandas
import numpy as np
import matplotlib.pyplot as plt
import seaborn
seaborn.set_style("whitegrid")
seaborn.despine()
%matplotlib inline
%config InlineBackend.figure_formats = {'svg',}

# Connect to the network
web3 = Web3(HTTPProvider('http://localhost:8545'))

# Get contract signature
with open('../ethereum/build/contracts/Market.json') as energy_file:    
    energy_json = json.load(energy_file)
contract_abi = energy_json['abi']
network_id = energy_json['networks'].keys()[-1]
contract_address = energy_json['networks'][network_id]['address']

# List accounts and set default account
accounts = web3.eth.accounts
web3.eth.defaultAccount = accounts[0]
contract = web3.eth.contract(contract_abi, contract_address)
energy_posted_event = contract.on('energy_posted_event', filter_params={'fromBlock': 'earliest'})
market_cleared_event = contract.on('market_cleared_event', filter_params={'fromBlock': 'earliest'})
bill_sent_event = contract.on('bill_sent_event', filter_params={'fromBlock': 'earliest'})

In [45]:
print('Number of accounts ' + str(len(accounts)))

# Add all participants
for account in accounts:
    trans_hash = contract.transact({'from': account, 'to': contract_address, 'gas': 900000}).add_participant()
    
# Check how many participants have been added
print('Number of participants ' + str(contract.call().number_of_participant()))

Number of accounts 10
Number of participants 10


In [46]:
# Inputs
energy_balances = [100, 500, 600, 200, 300, -100, -100, -200, -25, -50]
print('Total energy balance ' + str(sum(energy_balances)) + ' wh')
assert(len(accounts) == len(energy_balances))

# Post consumption and production for each account
for account, energy_balance in zip(accounts, energy_balances):
    temp = {'from': account, 'to': contract_address, 'gas': 900000}
    contract.transact(temp).post_energy_balance(int(energy_balance))

# Clear the market
contract.transact({'to': contract_address, 'gas': 900000}).clear_market()

# Print results

Total energy balance 1225 wh


u'0x9b6b8951c9b15a9166a10b45b24d5a1bf51820d1f79ad2ad616c02ea707b264b'

In [48]:
market_cleared_event.get(only_changes=False)

[{'address': u'0x8d56c2d21fce354dd2a13c0a4de8fd72c96b9bdd',
  'args': {u'_buy': 50,
   u'_gen': 115792089237316195423570985008687907853269984665640564039457584007913129638236L,
   u'_prod': 115792089237316195423570985008687907853269984665640564039457584007913129639461L,
   u'_sell': 50},
  'blockHash': u'0x99179e21b2ab825b8d25b0401937703f833ff1ac1aef2472cdaa4af01320f427',
  'blockNumber': 28,
  'event': u'market_cleared_event',
  'logIndex': 0,
  'transactionHash': u'0x9b6b8951c9b15a9166a10b45b24d5a1bf51820d1f79ad2ad616c02ea707b264b',
  'transactionIndex': 0}]

In [49]:
energy_posted_event.get(only_changes=False)

[{'address': u'0x8d56c2d21fce354dd2a13c0a4de8fd72c96b9bdd',
  'args': {u'_target': u'0x031667db452fc1a3decf0d0a45d1e4eeb572d47c',
   u'_value': 100},
  'blockHash': u'0xabea528b469ff8fedd5064e811fce29655998b4364ca0c4a55a41d8ae2fb4fe6',
  'blockNumber': 18,
  'event': u'energy_posted_event',
  'logIndex': 0,
  'transactionHash': u'0x08c0917a41a116307355b1ba8dbc305789c68578c278c89b92f7164be1e08466',
  'transactionIndex': 0},
 {'address': u'0x8d56c2d21fce354dd2a13c0a4de8fd72c96b9bdd',
  'args': {u'_target': u'0x584793145dc1e5daf0f587cbf436878587c8734c',
   u'_value': 500},
  'blockHash': u'0xdd687daece1ab28a90e70916adc9b39afca2ec5386a71636463a6c07aebd2c14',
  'blockNumber': 19,
  'event': u'energy_posted_event',
  'logIndex': 0,
  'transactionHash': u'0x3cf8b49abbbbfa2f2a28390242ee48885f655d6b78080e1a2f1da878b248bc60',
  'transactionIndex': 0},
 {'address': u'0x8d56c2d21fce354dd2a13c0a4de8fd72c96b9bdd',
  'args': {u'_target': u'0x6d32c15322b56a4fae1e526aae0c8ac25fa2552d',
   u'_value': 60

In [47]:
bill_sent_event.get(only_changes=False)

[{'address': u'0x8d56c2d21fce354dd2a13c0a4de8fd72c96b9bdd',
  'args': {u'_target': u'0x031667db452fc1a3decf0d0a45d1e4eeb572d47c',
   u'_value': 5000},
  'blockHash': u'0x99179e21b2ab825b8d25b0401937703f833ff1ac1aef2472cdaa4af01320f427',
  'blockNumber': 28,
  'event': u'bill_sent_event',
  'logIndex': 1,
  'transactionHash': u'0x9b6b8951c9b15a9166a10b45b24d5a1bf51820d1f79ad2ad616c02ea707b264b',
  'transactionIndex': 0},
 {'address': u'0x8d56c2d21fce354dd2a13c0a4de8fd72c96b9bdd',
  'args': {u'_target': u'0x584793145dc1e5daf0f587cbf436878587c8734c',
   u'_value': 25000},
  'blockHash': u'0x99179e21b2ab825b8d25b0401937703f833ff1ac1aef2472cdaa4af01320f427',
  'blockNumber': 28,
  'event': u'bill_sent_event',
  'logIndex': 2,
  'transactionHash': u'0x9b6b8951c9b15a9166a10b45b24d5a1bf51820d1f79ad2ad616c02ea707b264b',
  'transactionIndex': 0},
 {'address': u'0x8d56c2d21fce354dd2a13c0a4de8fd72c96b9bdd',
  'args': {u'_target': u'0x6d32c15322b56a4fae1e526aae0c8ac25fa2552d',
   u'_value': 30000},